In [3]:
import math, random

# Função conversora de binário para decimal
def convBinDec(b):
    d = 0
    for i in range(len(b)):
        d += int(b[i]) * (2 ** ((len(b) - 1) - i))
    return d

# Procedimento que quebra o endereço em Tag, Conjunto e Célula
def quebraEndereco():
    global endereco
    global endTag
    global endConjunto
    global endCelula
    for i in range(len(endereco)):
        if i >= 0 and i < tagBits:
            endTag = endTag + endereco[i] # Separa os bits da tag
        elif i >= tagBits and i < (tagBits + conjuntoBits):
            endConjunto = endConjunto + endereco[i] # Separa os bits do conjunto
        elif i >= (tagBits + conjuntoBits):
            endCelula = endCelula + endereco[i] # Separa os bits da celula/byte
    print("\nFormato de endereço da cache com mapeamento associativo por conjunto de",LpConj,":"+
              "\n| Tag      |",endTag,
              "\n| Conjunto |",endConjunto,
              "\n| Byte     |",endCelula)

# Função para buscar a tag na cache
def buscaTag():
    global iBusca # Índice de busca, se der Hit, ela vai estar com o índice em que a tag foi encontrada
    busca = False
    for iBusca in range((conjunto * LpConj), ((conjunto * LpConj) + LpConj)): #Laço que percorre as linhas do conjunto
        print("Linha",iBusca,":")
        print("| Tag |  Byte 1  |  Byte 0  |")
        exibeLinha(iBusca) # Chama o procedimento de saída de linha
        if endTag == cache[iBusca][len(cache[iBusca])-1]: # Verifica se a tag está na linha
            busca = True
            break
    return busca
    
# Procedimento para atualizar determinada linha da cache
def atualizaCache():
    global endTag
    global endConjunto
    global linha
    bloco = convBinDec(endTag+endConjunto) #Bloco da MP desejado
    print("\nBloco buscado na MP:", endTag+endConjunto)
    linha = random.randrange((conjunto * LpConj), ((conjunto * LpConj) + LpConj)) #Linha aleatória no conjunto
    print("Linha atualizada na Cache:", linha)
    j = 0
    for i in range((bloco * X), ((bloco * X) + X)): #Laço que percorre o bloco
        cache[linha][j] = mp[i] #Adiciona as células do bloco na linha da cache
        j += 1
    cache[linha][(len(cache[linha])-1)] = endTag #O último item da lista do cache[linha] recebe a tag

# Procedimento para saída dos dados da linha
def exibeLinha(l):
    saida = "| "
    for j in range(len(cache[l])-1, -1, -1):
        saida = saida + cache[l][j] + " | "
    print(saida)
    
mp = [] # Array da Memória Principal
Np = 32  # Capacidade de células da MP
X = 2  # Quantidade de células por bloco da MP/largura das linhas da cache
Ncache = 8  # Capacidade da cache
L = 4  # Quantidade de linhas da cache
LpConj = 2  # Quantidade de linha por conjunto

# Quantidade de blocos
B = int(Np / X)  # B = 16
# Quantidade de linhas
L = int(Ncache / X)  # L = 4
# Quantidade de conjuntos
C = int(L / LpConj)  # C = 2
# Quantidade bloco da MP por conjunto
BpConj = int(B / C)  # BpConj = 8
# Quantidade de bits da Tag
tagBits = int(math.log2(BpConj))  # tagBits = 3 bits
# Quantidade de bits do Conjunto
conjuntoBits = int(math.log2(C))  # conjuntoBits = 1 bit
# Quantidade de bits da Célula
celulaBits = int(math.log2(X))  # celulaBits = 1 bit

# Dados da Memória Cache: Byte 0 | Byte 1 | Tag
cache = []
cache.append(["00000000", "00000000", "000"])  # Linha 0 -- Conjunto 0 (Endereços 00000 e 00001)
cache.append(["00000000", "00000000", "001"])  # Linha 1 -- Conjunto 0 (Endereços 00100 e 00101)
cache.append(["00000000", "00000000", "000"])  # Linha 2 -- Conjunto 1 (Endereços 00010 e 00011)
cache.append(["00000000", "00000000", "001"])  # Linha 3 -- Conjunto 1 (Endereços 00110 e 00111)

print("\nMemória Cache\n| Tag |  Byte 1  |  Byte 0  |\n-----------------------------")
for i in range(len(cache)): # Formatação da saída das linhas da cache e listagem
    exibeLinha(i) # Chamada do procedimento

# Dados da Memória Principal
print("\nMemória Principal")
for i in range(Np):
    mp.append("00000000") #MP começa zerada
    print(mp[i])

# Operações
while True:
    print("\nSIMULADOR DE MEMÓRIA\n[W] Escrever\n[R] Ler\n[L] Listar")
    comando = input("Digite uma das opções ou qualquer tecla para parar: ") # Escolha das operações

    # ESCRITA
    if comando == "W" or comando == "w":
        # Entrada de endereço para escrita
        while True:
            try:
                endereco = input("Digite o endereço de 5 bits: ") # Verifica se o endereço possui apenas bits 0 e 1
                # Validação da entrada
                for i in range(len(endereco)):
                    if endereco[i] != "0" and endereco[i] != "1":
                        raise Exception
                        break
                if len(endereco) != 5: # Verifica se o endereço possui 5 bits
                    raise Exception
                break
            except Exception: # Caso o endereço não seja de bits de 0 e 1 e não tenha 5 bits
                print("Endereço inválido!")

        # Entrada de dados para escrita
        while True:
            try:
                dado = input("Digite o dado de 8 bits: ")
                # Validação da entrada
                for i in range(len(dado)):
                    if dado[i] != "0" and dado[i] != "1": # Verifica se o dado possui apenas bits 0 e 1
                        raise Exception
                        break
                if len(dado) != 8: # Verifica se o dado possui 8 bits
                    raise Exception
                break
            except Exception: # Caso o dado não seja de bits de 0 e 1 e não tenha 8 bits
                print("Dado inválido!")
        
        
        # Quebra do endereço
        endTag = ""
        endConjunto = ""
        endCelula = ""
        quebraEndereco() # Chamada do procedimento  
        
        #Escrita na Cache
        conjunto = convBinDec(endConjunto) #Conversão do bit do binário para decimal
        print("\nEscrita no conjunto",conjunto,"da Cache:")
         
        if(buscaTag() == True): # Atribui a linha dependendo da tag estar ou não no conjunto, chamando a função de busca
            print("Bloco "+endTag+endConjunto+" encontrado na Cache")
            linha = iBusca # A linha recebe a posição da linha na cache em que a tag foi encontrada
        else:
            print("Bloco "+endTag+endConjunto+" não encontrado na Cache")
            atualizaCache() # Chamada do procedimento
        
        cache[linha][convBinDec(endCelula)] = dado # A célula atualizada recebe o dado novo
        print("\nLinha", linha, "<--- " + dado)
        print("| Tag |  Byte 1  |  Byte 0  |")
        exibeLinha(linha) # Chamada do procedimento
        
        #Escrita na Memória Principal
        print("\nEscrita na Memória Principal\nMP <--- Cache")
        mp[convBinDec(endereco)] = cache[linha][convBinDec(endCelula)] # Cache envia célula para MP conforme o endereço inserido
        print("Endereço " + endereco + " <--- " + cache[linha][convBinDec(endCelula)])
        print("")
          

    # LEITURA
    elif comando == "R" or comando == "r":
        # Entrada de endereço para leitura
        while True:
            try:
                endereco = input("Digite o endereço de 5 bits: ") # Verifica se o endereço possui apenas bits 0 e 1
                # Validação da entrada
                for i in range(len(endereco)):
                    if endereco[i] != "0" and endereco[i] != "1":
                        raise Exception
                        break
                if len(endereco) != 5: # Verifica se o endereço possui 5 bits
                    raise Exception
                break
            except Exception: # Caso o endereço não seja de bits de 0 e 1 e não tenha 5 bits
                print("Endereço inválido!")

        # Quebra do endereço
        endTag = ""
        endConjunto = ""
        endCelula = ""
        quebraEndereco() # Chamada do procedimento  

        # Busca da tag na cache
        print("\nAcesso ao endereço na Memória Cache para Leitura")
        conjunto = convBinDec(endConjunto) #Conversão do bit do binário para decimal
        print("Busca no Conjunto",conjunto)
       
        if buscaTag() == True:  #Se a busca der Hit, chamando a função que faz a busca
            print("Resultado: Hit")
            linha = iBusca # A linha recebe a posição da linha na cache em que a tag foi encontrada
            print("\nInformação na Cache:\nEndereço: " + endereco + "\nDado: " + cache[linha][convBinDec(endCelula)])
            print("")
        else: #Se a busca der Miss
            print("Resultado: Miss\nO bloco será buscado na Memória Principal")
            atualizaCache() # Chamada do procedimento
            print("| Tag |  Byte 1  |  Byte 0  |")
            exibeLinha(linha) # Chamada do procedimento
            print("\nEndereço: " + endereco + "\nDado: " + cache[linha][convBinDec(endCelula)])
            
            
    # LISTAGEM
    elif comando == "L" or comando == "l":
        print("\nMemória Cache:\n| Tag |  Byte 1  |  Byte 0  |\n-----------------------------")
        for i in range(len(cache)): # Formatação da saída das linhas da cache e listagem
            exibeLinha(i) # Chamada do procedimento
        
        print("\nMemória Principal:") #Listagem da MP
        for i in range(Np):
            print(mp[i])
            
    # Encerramento
    else:
        print("Programa encerrado")
        break


Memória Cache
| Tag |  Byte 1  |  Byte 0  |
-----------------------------
| 000 | 00000000 | 00000000 | 
| 001 | 00000000 | 00000000 | 
| 000 | 00000000 | 00000000 | 
| 001 | 00000000 | 00000000 | 

Memória Principal
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000
00000000

SIMULADOR DE MEMÓRIA
[W] Escrever
[R] Ler
[L] Listar
Digite uma das opções ou qualquer tecla para parar: W
Digite o endereço de 5 bits: 110010
Endereço inválido!
Digite o endereço de 5 bits: 1=-1
Endereço inválido!
Digite o endereço de 5 bits: 11101
Digite o dado de 8 bits: 11110000

Formato de endereço da cache com mapeamento associativo por conjunto de 2 :
| Tag      | 111 
| Conjunto | 0 
| Byte     | 1

Escrita no conjunto 0 da Cache:
Linha 0 :
| Tag |  Byte 1  |  Byte 0  |
| 000 | 00